In [1]:
import os
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split
from map_boxes import mean_average_precision_for_boxes
import pandas as pd
import random

In [2]:
!git clone https://github.com/ultralytics/yolov5.git

Клонирование в «yolov5»…
remote: Enumerating objects: 13178, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 13178 (delta 30), reused 27 (delta 13), pack-reused 13119
Получение объектов: 100% (13178/13178), 12.56 МиБ | 8.41 МиБ/с, готово.
Определение изменений: 100% (9042/9042), готово.


In [3]:
!pip install -r yolov5/requirements.txt

In [2]:
len(os.listdir('test/images')), len(os.listdir('train/images'))

(855, 2002)

In [3]:
#распределение объектов в train
pd.Series([file_name.split('_')[-1] for file_name in os.listdir('train/labels')]).value_counts()

head..txt        1741
human..txt       1740
car..txt         1647
face..txt         630
carplate..txt     246
dtype: int64

In [7]:
#создадим обучающий датасет в формате yolo
!mkdir data_for_yolo
!mkdir data_for_yolo/data
!mkdir data_for_yolo/data/images
!mkdir data_for_yolo/data/labels
!mkdir data_for_yolo/data/images/train
!mkdir data_for_yolo/data/labels/train
!mkdir data_for_yolo/data/images/test
!mkdir data_for_yolo/data/labels/test

In [8]:
#в папке yolov5 создадим dataset.yaml со следующим содержимым

yaml_content = """
train: ../data_for_yolo/data/images/train/
val: ../data_for_yolo/data/images/test/

# number of classes
nc: 1

# class names
names: ['human']
"""

In [9]:
human_labels_only = [file_name for file_name in os.listdir('train/labels') if 'human..txt' in file_name]
print(len(human_labels_only))

1740


In [10]:
human_images_names = [file_name.split('_human..txt')[0] for file_name in human_labels_only]
print(len(human_images_names))

1740


In [11]:
train_images_names, test_images_names = train_test_split(human_images_names, test_size=0.2, random_state=22)
print(len(train_images_names), len(test_images_names))

1392 348


In [12]:
#train
for train_image in train_images_names:
    #labels
    for file in os.listdir('train/labels'): 
        if '_human..txt' in file and train_image in file: 
            shutil.copy('train/labels/' + file, 'data_for_yolo/data/labels/train/' + file.replace('_human..txt', '.txt'))
        #images 
    for file in os.listdir('train/images'): 
        if train_image + '.jpg' == file: 
            shutil.copy('train/images/' + file, 'data_for_yolo/data/images/train/')

#test
for test_image in test_images_names:
    #labels
    for file in os.listdir('train/labels'): 
        if '_human..txt' in file and test_image in file: 
            shutil.copy('train/labels/' + file, 'data_for_yolo/data/labels/test/' + file.replace('_human..txt', '.txt'))
    #images 
    for file in os.listdir('train/images'): 
        if test_image + '.jpg' == file: 
            shutil.copy('train/images/' + file, 'data_for_yolo/data/images/test/')

In [4]:
#валидация 
print(len(os.listdir('data_for_yolo/data/images/test')))
print(len(os.listdir('data_for_yolo/data/images/train')))

348
1392


In [5]:
print(len(os.listdir('data_for_yolo/data/labels/test')))
print(len(os.listdir('data_for_yolo/data/labels/train')))

348
1392


# Обучение - human

In [6]:
path_to_data = 'dataset.yaml'

In [7]:
!nvidia-smi

Mon Aug  1 12:25:33 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.80.02    Driver Version: 450.80.02    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 105...  Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   47C    P8    N/A /  N/A |    394MiB /  4042MiB |     19%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#--img 150 --batch 32 --epochs 5 --data dataset.yaml --weights yolov5s.pt

In [8]:
!cd yolov5 && python3 train.py --img 416 --batch 16 --epochs 100 --data {path_to_data} --cfg models/yolov5s.yaml --name yolov5s_results --cache

train: weights=yolov5s.pt, cfg=models/yolov5s.yaml, data=dataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=yolov5s_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
remote: Enumerating objects: 116, done.
remote: Counting objects: 100% (114/114), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 116 (delta 63), reused 81 (delta 38), pack-reused 2
Получение объектов: 100% (116/116), 111.57 КиБ | 7.97 МиБ/с, готово.
Определение изменений: 100% (63/63), завершено с 4 локальными объектами.
Из https://github.com/ultralytics/yo

               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        348        920      0.715      0.445      0.523      0.209

     Epoch   gpu_mem       box       obj       cls    labels  img_size
      5/99      1.7G   0.08524   0.01995         0        81       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        348        920      0.698      0.497      0.553      0.238

     Epoch   gpu_mem       box       obj       cls    labels  img_size
      6/99      1.7G   0.08302   0.01954         0        65       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        348        920      0.694      0.458      0.535       0.23

     Epoch   gpu_mem       box       obj       cls    labels  img_size
      7/99      1.7G   0.08264   0.02002         0        51       416: 100%|███
               Class     Images     L


     Epoch   gpu_mem       box       obj       cls    labels  img_size
     30/99      1.7G   0.06719   0.01708         0        61       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        348        920      0.848      0.593      0.686      0.354

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     31/99      1.7G    0.0675   0.01718         0        69       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        348        920      0.822      0.596      0.675       0.35

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     32/99      1.7G   0.06687   0.01717         0        67       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        348        920       0.85       0.59      0.676      0.336

     Epoch   gpu_mem       box      

               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        348        920      0.876      0.617      0.721      0.383

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     56/99      1.7G   0.06233   0.01622         0        61       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        348        920      0.814      0.628      0.705      0.373

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     57/99      1.7G   0.06296   0.01595         0        82       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        348        920      0.857      0.625      0.712      0.378

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     58/99      1.7G   0.05811   0.01542         0        59       416: 100%|███
               Class     Images     L


     Epoch   gpu_mem       box       obj       cls    labels  img_size
     81/99      1.7G    0.0576   0.01453         0        61       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        348        920      0.839      0.647       0.73      0.401

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     82/99      1.7G   0.05737   0.01468         0        62       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        348        920      0.897      0.624      0.734      0.402

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     83/99      1.7G    0.0575   0.01473         0        43       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        348        920      0.844      0.647      0.727      0.396

     Epoch   gpu_mem       box      

In [126]:
!cd yolov5 && python3 detect.py --weights runs/train/yolov5s_results3/weights/best.pt --img 416 --conf 0.4 --source ../test/images --save-txt --save-conf

detect: weights=['runs/train/yolov5s_results3/weights/best.pt'], source=../test/images, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-328-ge309a85 Python-3.7.11 torch-1.7.0 CUDA:0 (GeForce GTX 1050 Ti, 4042MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/0_23-11-2021_02-18-50_PM.jpg: 288x416 Done. (0.009s)
image 2/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/0_23-11-2021_02-18-56_PM.jpg: 352x416 1 human, Done. (0.008s)
image 3/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_20

image 53/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/0_25-11-2021_08-09-32_PM.jpg: 320x416 Done. (0.009s)
image 54/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/0_25-11-2021_08-34-03_AM.jpg: 320x416 Done. (0.009s)
image 55/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/0_25-11-2021_08-50-08_AM.jpg: 288x416 1 human, Done. (0.008s)
image 56/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/0_25-11-2021_09-27-14_AM.jpg: 288x416 Done. (0.007s)
image 57/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/0_25-11-2021_09-46-26_AM.jpg: 320x416 Done. (0.008s)
image 58/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/0_25-11-2021_10-03-07_AM.jpg: 288x416 Done. (0.009s)
image 59/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/0_25-11-2021_10-03-50_AM.jpg: 224x41

image 110/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_02-21-18_PM.jpg: 256x416 3 humans, Done. (0.007s)
image 111/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_02-36-28_PM.jpg: 256x416 1 human, Done. (0.008s)
image 112/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_02-37-17_PM.jpg: 256x416 3 humans, Done. (0.007s)
image 113/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_02-37-22_PM.jpg: 256x416 1 human, Done. (0.008s)
image 114/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_02-37-31_PM.jpg: 256x416 3 humans, Done. (0.008s)
image 115/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_02-37-35_PM.jpg: 256x416 1 human, Done. (0.007s)
image 116/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/

image 165/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_05-02-16_PM.jpg: 256x416 1 human, Done. (0.009s)
image 166/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_05-18-06_PM.jpg: 256x416 2 humans, Done. (0.009s)
image 167/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_05-18-46_PM.jpg: 256x416 Done. (0.009s)
image 168/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_05-19-24_PM.jpg: 256x416 1 human, Done. (0.008s)
image 169/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_05-35-51_PM.jpg: 256x416 2 humans, Done. (0.008s)
image 170/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_05-36-00_PM.jpg: 256x416 3 humans, Done. (0.007s)
image 171/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23

image 220/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/24-11-2021_02-25-31_PM.jpg: 256x416 2 humans, Done. (0.009s)
image 221/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/24-11-2021_02-25-34_PM.jpg: 256x416 1 human, Done. (0.009s)
image 222/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/24-11-2021_02-25-41_PM.jpg: 256x416 3 humans, Done. (0.009s)
image 223/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/24-11-2021_02-25-48_PM.jpg: 256x416 2 humans, Done. (0.009s)
image 224/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/24-11-2021_02-31-10_AM.jpg: 256x416 1 human, Done. (0.011s)
image 225/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/24-11-2021_02-31-12_AM.jpg: 256x416 Done. (0.009s)
image 226/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/24

...

image 599/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_07-03-30_AM.jpg: 256x416 1 human, Done. (0.008s)
image 600/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_07-03-59_AM.jpg: 256x416 1 human, Done. (0.014s)
image 601/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_07-04-28_AM.jpg: 256x416 3 humans, Done. (0.008s)
image 602/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_07-04-51_AM.jpg: 256x416 7 humans, Done. (0.009s)
image 603/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_07-05-01_AM.jpg: 256x416 2 humans, Done. (0.008s)
image 604/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_07-15-03_PM.jpg: 256x416 1 human, Done. (0.009s)
image 605/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/

image 653/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_09-25-39_AM.jpg: 256x416 2 humans, Done. (0.008s)
image 654/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_09-26-33_AM.jpg: 256x416 2 humans, Done. (0.008s)
image 655/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_09-26-47_AM.jpg: 256x416 1 human, Done. (0.009s)
image 656/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_09-26-52_AM.jpg: 256x416 1 human, Done. (0.009s)
image 657/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_09-27-38_AM.jpg: 256x416 1 human, Done. (0.008s)
image 658/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_09-44-25_AM.jpg: 256x416 2 humans, Done. (0.007s)
image 659/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/

image 708/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_03-54-45_AM.jpg: 256x416 1 human, Done. (0.007s)
image 709/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_03-54-53_AM.jpg: 256x416 1 human, Done. (0.011s)
image 710/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_03-55-17_AM.jpg: 256x416 1 human, Done. (0.011s)
image 711/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_04-10-47_AM.jpg: 256x416 2 humans, Done. (0.009s)
image 712/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_04-11-09_AM.jpg: 256x416 1 human, Done. (0.010s)
image 713/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_04-11-13_AM.jpg: 256x416 Done. (0.009s)
image 714/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-1

image 763/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_07-11-54_AM.jpg: 256x416 2 humans, Done. (0.009s)
image 764/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_07-27-54_AM.jpg: 256x416 1 human, Done. (0.009s)
image 765/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_07-29-09_AM.jpg: 256x416 1 human, Done. (0.008s)
image 766/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_07-29-39_AM.jpg: 256x416 1 human, Done. (0.010s)
image 767/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_07-29-48_AM.jpg: 256x416 1 human, Done. (0.009s)
image 768/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_07-30-30_AM.jpg: 320x416 Done. (0.008s)
image 769/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-1

image 818/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_10-10-07_AM.jpg: 256x416 1 human, Done. (0.007s)
image 819/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_10-10-13_AM.jpg: 256x416 3 humans, Done. (0.008s)
image 820/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_10-10-46_AM.jpg: 256x416 1 human, Done. (0.009s)
image 821/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_10-11-07_AM.jpg: 256x416 1 human, Done. (0.008s)
image 822/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_10-28-02_AM.jpg: 256x416 1 human, Done. (0.008s)
image 823/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_10-28-51_AM.jpg: 256x416 6 humans, Done. (0.007s)
image 824/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/i

In [127]:
def get_soliton_labels_df(path_to_txt_folder):
    simple_solution = []
    for detection_file in os.listdir(path_to_txt_folder):
        img_name = detection_file.split('.')[0] + '.jpg'
        with open(path_to_txt_folder + detection_file, 'r') as f:
            data = f.read()
            data = [i for i in data.split('\n') if i != '']
        for line in data:
            val = [float(i) for i in line.split()]
            cls, xywh, conf = val[0], val[1:5], val[5]
            center_x, center_y, width, height = xywh
            xmin = center_x - (width / 2)
            xmax = center_x + (width / 2)
            ymin = center_y - (height / 2)
            ymax = center_y + (height / 2)
            simple_solution.append([img_name, cls, conf, xmin, xmax, ymin, ymax])
    return simple_solution

In [128]:
simple_solution = get_soliton_labels_df('yolov5/runs/detect/exp7/labels/')
simple_solution = pd.DataFrame(simple_solution, columns=['ImageID', 'LabelName', 'Conf', 'XMin', 'XMax', 'YMin', 'YMax'])

In [129]:
simple_solution

,ImageID,LabelName,Conf,XMin,XMax,YMin,YMax
0,24-11-2021_11-08-42_AM.jpg,0.0,0.661521,0.113021,0.127083,0.002778,0.061111
1,24-11-2021_11-08-42_AM.jpg,0.0,0.899546,0.266146,0.310416,0.252777,0.390741
2,24-11-2021_05-30-07_AM.jpg,0.0,0.769015,0.164062,0.176042,0.233333,0.287963
3,24-11-2021_02-43-19_PM.jpg,0.0,0.891145,0.170139,0.222223,0.294754,0.523149
4,0_23-11-2021_02-18-56_PM.jpg,0.0,0.414341,0.008333,0.179167,0.350000,0.695000
...,...,...,...,...,...,...,...
1486,25-11-2021_04-18-04_PM.jpg,0.0,0.899927,0.805208,0.854688,0.518518,0.672222
1487,24-11-2021_12-20-59_PM.jpg,0.0,0.838950,0.406250,0.423958,0.133333,0.212037
1488,24-11-2021_12-20-59_PM.jpg,0.0,0.877806,0.024479,0.049479,0.213889,0.362037
1489,26-11-2021_04-12-57_AM.jpg,0.0,0.904862,0.641146,0.677604,0.303704,0.440740


In [130]:
simple_solution2 = simple_solution.copy()

In [131]:
simple_solution2['LabelName'] = 3

In [132]:
simple_solution2

,ImageID,LabelName,Conf,XMin,XMax,YMin,YMax
0,24-11-2021_11-08-42_AM.jpg,3,0.661521,0.113021,0.127083,0.002778,0.061111
1,24-11-2021_11-08-42_AM.jpg,3,0.899546,0.266146,0.310416,0.252777,0.390741
2,24-11-2021_05-30-07_AM.jpg,3,0.769015,0.164062,0.176042,0.233333,0.287963
3,24-11-2021_02-43-19_PM.jpg,3,0.891145,0.170139,0.222223,0.294754,0.523149
4,0_23-11-2021_02-18-56_PM.jpg,3,0.414341,0.008333,0.179167,0.350000,0.695000
...,...,...,...,...,...,...,...
1486,25-11-2021_04-18-04_PM.jpg,3,0.899927,0.805208,0.854688,0.518518,0.672222
1487,24-11-2021_12-20-59_PM.jpg,3,0.838950,0.406250,0.423958,0.133333,0.212037
1488,24-11-2021_12-20-59_PM.jpg,3,0.877806,0.024479,0.049479,0.213889,0.362037
1489,26-11-2021_04-12-57_AM.jpg,3,0.904862,0.641146,0.677604,0.303704,0.440740


In [133]:
simple_solution2.to_csv("submit_human.csv", sep=';', index=False)

# Считаем индекс 0 - car

In [29]:
car_labels_only = [file_name for file_name in os.listdir('train/labels') if 'car..txt' in file_name]
print(len(car_labels_only))

1647


In [30]:
car_images_names = [file_name.split('_car..txt')[0] for file_name in car_labels_only]
print(len(car_images_names))

1647


In [31]:
train_images_names, test_images_names = train_test_split(car_images_names, test_size=0.2, random_state=22)
print(len(train_images_names), len(test_images_names))

1317 330


In [32]:
#создадим обучающий датасет в формате yolo
!mkdir data_for_yolo/data_car
!mkdir data_for_yolo/data_car/images
!mkdir data_for_yolo/data_car/labels
!mkdir data_for_yolo/data_car/images/train
!mkdir data_for_yolo/data_car/labels/train
!mkdir data_for_yolo/data_car/images/test
!mkdir data_for_yolo/data_car/labels/test

mkdir: невозможно создать каталог «data_for_yolo/data_car»: Файл существует
mkdir: невозможно создать каталог «data_for_yolo/data_car/images»: Файл существует
mkdir: невозможно создать каталог «data_for_yolo/data_car/labels»: Файл существует
mkdir: невозможно создать каталог «data_for_yolo/data_car/images/train»: Файл существует
mkdir: невозможно создать каталог «data_for_yolo/data_car/labels/train»: Файл существует
mkdir: невозможно создать каталог «data_for_yolo/data_car/images/test»: Файл существует
mkdir: невозможно создать каталог «data_for_yolo/data_car/labels/test»: Файл существует


In [33]:
#train
for train_image in train_images_names:
    #labels
    for file in os.listdir('train/labels'): 
        if '_car..txt' in file and train_image in file: 
            shutil.copy('train/labels/' + file, 'data_for_yolo/data_car/labels/train/' + file.replace('_car..txt', '.txt'))
        #images 
    for file in os.listdir('train/images'): 
        if train_image + '.jpg' == file: 
            shutil.copy('train/images/' + file, 'data_for_yolo/data_car/images/train/')

#test
for test_image in test_images_names:
    #labels
    for file in os.listdir('train/labels'): 
        if '_car..txt' in file and test_image in file: 
            shutil.copy('train/labels/' + file, 'data_for_yolo/data_car/labels/test/' + file.replace('_car..txt', '.txt'))
    #images 
    for file in os.listdir('train/images'): 
        if test_image + '.jpg' == file: 
            shutil.copy('train/images/' + file, 'data_for_yolo/data_car/images/test/')

In [34]:
#валидация 
print(len(os.listdir('data_for_yolo/data_car/images/test')))
print(len(os.listdir('data_for_yolo/data_car/images/train')))

330
1317


In [35]:
print(len(os.listdir('data_for_yolo/data_car/labels/test')))
print(len(os.listdir('data_for_yolo/data_car/labels/train')))

330
1317


## Обучение - car

In [36]:
path_to_data = 'dataset_car.yaml'

In [37]:
!cd yolov5 && python3 train.py --img 416 --batch 16 --epochs 100 --data {path_to_data} --cfg models/yolov5s.yaml --name yolov5s_results --cache

train: weights=yolov5s.pt, cfg=models/yolov5s.yaml, data=dataset_car.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=yolov5s_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 20 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.1-328-ge309a85 Python-3.7.11 torch-1.7.0 CUDA:0 (GeForce GTX 1050 Ti, 4042MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_p

train: Caching images (0.4GB ram): 100%|██████████| 1317/1317 [00:08<00:00, 155.
val: Scanning '/home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан
val: WARNING: /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/yolov5/../data_for_yolo/data_car/images/test/24-11-2021_01-49-49_PM.jpg: 1 duplicate labels removed
val: WARNING: /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/yolov5/../data_for_yolo/data_car/images/test/24-11-2021_04-36-08_AM.jpg: 3 duplicate labels removed
val: WARNING: /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/yolov5/../data_for_yolo/data_car/images/test/24-11-2021_05-30-51_AM.jpg: 1 duplicate labels removed
val: WARNING: /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/yolov5/../data_for_yolo/data_car/images/test/24-11-2021_11-27-09_AM.jpg: 2 duplicate labels removed
val: WARNING: /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/yolov5/../data_for_yolo/data_car/image

     19/99     1.75G   0.04083   0.04395         0       200       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        330       3162      0.897       0.82      0.904      0.611

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     20/99     1.75G   0.04026   0.04282         0        47       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        330       3162      0.899      0.804      0.898      0.587

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     21/99     1.75G   0.04019   0.04233         0       125       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        330       3162      0.905      0.809      0.899      0.586

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     22/99     1.75G   0.03948   0.04


     Epoch   gpu_mem       box       obj       cls    labels  img_size
     45/99     1.75G   0.03534   0.03769         0        80       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        330       3162      0.918      0.838      0.928      0.662

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     46/99     1.75G   0.03574    0.0391         0       101       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        330       3162      0.912      0.844      0.925      0.664

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     47/99     1.75G   0.03541   0.03738         0       135       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        330       3162      0.925      0.831      0.924       0.66

     Epoch   gpu_mem       box      

               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        330       3162       0.93      0.845      0.936      0.687

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     71/99     1.75G   0.03328   0.03625         0       117       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        330       3162      0.925      0.848      0.933      0.684

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     72/99     1.75G   0.03312   0.03553         0        78       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        330       3162       0.93      0.842      0.931      0.685

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     73/99     1.75G   0.03331   0.03569         0       181       416: 100%|███
               Class     Images     L


     Epoch   gpu_mem       box       obj       cls    labels  img_size
     96/99     1.75G   0.03244   0.03438         0       122       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        330       3162      0.921       0.86      0.937      0.699

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     97/99     1.75G   0.03189   0.03384         0        76       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        330       3162      0.912      0.866      0.937        0.7

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     98/99     1.75G   0.03171   0.03332         0        63       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        330       3162      0.916      0.864      0.937        0.7

     Epoch   gpu_mem       box      

In [134]:
# Поправить yolov5s_results3 ?
!cd yolov5 && python3 detect.py --weights runs/train/yolov5s_results5/weights/best.pt --img 416 --conf 0.4 --source ../test/images --save-txt --save-conf

detect: weights=['runs/train/yolov5s_results5/weights/best.pt'], source=../test/images, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-328-ge309a85 Python-3.7.11 torch-1.7.0 CUDA:0 (GeForce GTX 1050 Ti, 4042MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/0_23-11-2021_02-18-50_PM.jpg: 288x416 Done. (0.008s)
image 2/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/0_23-11-2021_02-18-56_PM.jpg: 352x416 Done. (0.008s)
image 3/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарс

image 57/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/0_25-11-2021_09-46-26_AM.jpg: 320x416 1 car, Done. (0.009s)
image 58/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/0_25-11-2021_10-03-07_AM.jpg: 288x416 Done. (0.008s)
image 59/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/0_25-11-2021_10-03-50_AM.jpg: 224x416 1 car, Done. (0.008s)
image 60/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/0_25-11-2021_10-55-30_AM.jpg: 352x416 Done. (0.009s)
image 61/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/0_25-11-2021_11-14-06_AM.jpg: 352x416 Done. (0.008s)
image 62/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/0_26-11-2021_02-25-08_AM.jpg: 320x416 Done. (0.008s)
image 63/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/0_26-11-2021_03-53-12_AM.jpg: 3

image 114/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_02-37-31_PM.jpg: 256x416 8 cars, Done. (0.007s)
image 115/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_02-37-35_PM.jpg: 256x416 8 cars, Done. (0.007s)
image 116/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_02-37-57_PM.jpg: 256x416 24 cars, Done. (0.009s)
image 117/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_02-38-55_PM.jpg: 256x416 Done. (0.007s)
image 118/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_02-39-07_PM.jpg: 320x416 2 cars, Done. (0.009s)
image 119/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_02-54-50_PM.jpg: 256x416 3 cars, Done. (0.008s)
image 120/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-202

image 169/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_05-35-51_PM.jpg: 256x416 19 cars, Done. (0.007s)
image 170/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_05-36-00_PM.jpg: 256x416 16 cars, Done. (0.007s)
image 171/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_05-36-22_PM.jpg: 256x416 10 cars, Done. (0.007s)
image 172/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_05-37-26_PM.jpg: 256x416 1 car, Done. (0.008s)
image 173/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_05-38-10_PM.jpg: 256x416 1 car, Done. (0.010s)
image 174/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_05-55-09_PM.jpg: 256x416 10 cars, Done. (0.012s)
image 175/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/

image 224/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/24-11-2021_02-31-10_AM.jpg: 256x416 2 cars, Done. (0.008s)
image 225/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/24-11-2021_02-31-12_AM.jpg: 256x416 Done. (0.008s)
image 226/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/24-11-2021_02-32-58_AM.jpg: 256x416 1 car, Done. (0.007s)
image 227/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/24-11-2021_02-33-05_AM.jpg: 256x416 3 cars, Done. (0.009s)
image 228/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/24-11-2021_02-42-12_PM.jpg: 256x416 4 cars, Done. (0.009s)
image 229/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/24-11-2021_02-42-46_PM.jpg: 256x416 25 cars, Done. (0.008s)
image 230/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/24-11-2021

...

image 609/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_07-33-52_PM.jpg: 256x416 24 cars, Done. (0.009s)
image 610/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_07-34-00_PM.jpg: 256x416 30 cars, Done. (0.008s)
image 611/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_07-34-32_PM.jpg: 256x416 2 cars, Done. (0.009s)
image 612/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_07-56-34_AM.jpg: 256x416 Done. (0.009s)
image 613/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_07-57-05_AM.jpg: 256x416 15 cars, Done. (0.010s)
image 614/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_07-57-51_AM.jpg: 256x416 21 cars, Done. (0.008s)
image 615/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-

image 664/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_10-01-52_AM.jpg: 256x416 2 cars, Done. (0.008s)
image 665/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_10-02-28_AM.jpg: 256x416 8 cars, Done. (0.008s)
image 666/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_10-02-36_AM.jpg: 256x416 16 cars, Done. (0.009s)
image 667/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_10-02-51_AM.jpg: 256x416 14 cars, Done. (0.008s)
image 668/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_10-03-12_AM.jpg: 256x416 Done. (0.009s)
image 669/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_10-03-33_AM.jpg: 256x416 10 cars, Done. (0.009s)
image 670/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2

image 719/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_04-29-31_AM.jpg: 256x416 18 cars, Done. (0.007s)
image 720/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_04-29-34_AM.jpg: 256x416 10 cars, Done. (0.009s)
image 721/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_04-30-50_AM.jpg: 256x416 1 car, Done. (0.008s)
image 722/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_04-47-01_AM.jpg: 256x416 9 cars, Done. (0.007s)
image 723/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_04-47-28_AM.jpg: 256x416 7 cars, Done. (0.009s)
image 724/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_04-47-47_AM.jpg: 256x416 26 cars, Done. (0.008s)
image 725/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/

image 774/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_07-46-51_AM.jpg: 256x416 3 cars, Done. (0.009s)
image 775/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_07-47-56_AM.jpg: 256x416 24 cars, Done. (0.007s)
image 776/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_07-48-27_AM.jpg: 256x416 14 cars, Done. (0.009s)
image 777/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_08-03-58_AM.jpg: 256x416 5 cars, Done. (0.011s)
image 778/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_08-04-15_AM.jpg: 256x416 10 cars, Done. (0.008s)
image 779/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_08-05-04_AM.jpg: 256x416 14 cars, Done. (0.008s)
image 780/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/image

image 829/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_11-22-36_AM.jpg: 256x416 18 cars, Done. (0.008s)
image 830/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_11-25-09_AM.jpg: 256x416 14 cars, Done. (0.007s)
image 831/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_11-42-01_AM.jpg: 256x416 7 cars, Done. (0.007s)
image 832/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_11-42-06_AM.jpg: 256x416 4 cars, Done. (0.008s)
image 833/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_11-44-10_AM.jpg: 256x416 Done. (0.008s)
image 834/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_11-44-35_AM.jpg: 256x416 17 cars, Done. (0.009s)
image 835/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2

In [135]:
# Поменять путь yolov5/runs/detect/exp2/labels/ !!!
simple_solution_car = get_soliton_labels_df('yolov5/runs/detect/exp8/labels/')
simple_solution_car = pd.DataFrame(simple_solution_car, columns=['ImageID', 'LabelName', 'Conf', 'XMin', 'XMax', 'YMin', 'YMax'])

In [136]:
simple_solution_car

,ImageID,LabelName,Conf,XMin,XMax,YMin,YMax
0,24-11-2021_11-08-42_AM.jpg,0.0,0.662121,0.960938,1.000000,0.232407,0.439814
1,24-11-2021_11-08-42_AM.jpg,0.0,0.914958,0.636459,0.834896,0.180555,0.411111
2,24-11-2021_11-08-42_AM.jpg,0.0,0.940789,0.391146,0.545312,0.027778,0.192592
3,24-11-2021_11-08-42_AM.jpg,0.0,0.946185,0.187499,0.485416,0.354630,0.674074
4,24-11-2021_11-08-42_AM.jpg,0.0,0.956870,0.259896,0.410938,0.111110,0.302777
...,...,...,...,...,...,...,...
7312,25-11-2021_04-18-04_PM.jpg,0.0,0.939549,0.506771,0.681771,0.404629,0.558333
7313,25-11-2021_04-18-04_PM.jpg,0.0,0.949648,0.310416,0.443750,0.492592,0.710185
7314,25-11-2021_04-18-04_PM.jpg,0.0,0.956506,0.556771,0.746355,0.446296,0.628703
7315,24-11-2021_12-20-59_PM.jpg,0.0,0.964550,0.671875,0.896355,0.425926,0.675926


In [137]:
simple_solution_car['LabelName'] = str(0)

In [138]:
simple_solution_car 

,ImageID,LabelName,Conf,XMin,XMax,YMin,YMax
0,24-11-2021_11-08-42_AM.jpg,0,0.662121,0.960938,1.000000,0.232407,0.439814
1,24-11-2021_11-08-42_AM.jpg,0,0.914958,0.636459,0.834896,0.180555,0.411111
2,24-11-2021_11-08-42_AM.jpg,0,0.940789,0.391146,0.545312,0.027778,0.192592
3,24-11-2021_11-08-42_AM.jpg,0,0.946185,0.187499,0.485416,0.354630,0.674074
4,24-11-2021_11-08-42_AM.jpg,0,0.956870,0.259896,0.410938,0.111110,0.302777
...,...,...,...,...,...,...,...
7312,25-11-2021_04-18-04_PM.jpg,0,0.939549,0.506771,0.681771,0.404629,0.558333
7313,25-11-2021_04-18-04_PM.jpg,0,0.949648,0.310416,0.443750,0.492592,0.710185
7314,25-11-2021_04-18-04_PM.jpg,0,0.956506,0.556771,0.746355,0.446296,0.628703
7315,24-11-2021_12-20-59_PM.jpg,0,0.964550,0.671875,0.896355,0.425926,0.675926


In [139]:
simple_solution_car.to_csv("submit_car.csv", sep=';', index=False)

In [140]:
len(simple_solution2)

1491

In [141]:
simple_solution30 = simple_solution2.append(simple_solution_car, ignore_index = True)

In [142]:
len(simple_solution30)

8808

In [49]:
simple_solution30[670:750]

,ImageID,LabelName,Conf,XMin,XMax,YMin,YMax
670,25-11-2021_11-13-23_AM.jpg,3,0.890784,0.268229,0.311459,0.596296,0.743518
671,25-11-2021_11-13-23_AM.jpg,3,0.932812,0.501563,0.538021,0.606481,0.774074
672,25-11-2021_03-58-56_PM.jpg,3,0.695209,0.392113,0.403645,0.201315,0.255263
673,25-11-2021_03-58-56_PM.jpg,3,0.892723,0.494792,0.528646,0.386842,0.528289
674,24-11-2021_04-36-37_AM.jpg,3,0.775627,0.144792,0.162500,0.362037,0.430555
...,...,...,...,...,...,...,...
745,24-11-2021_04-18-24_AM.jpg,0,0.743848,0.003646,0.057813,0.122222,0.173148
746,24-11-2021_04-18-24_AM.jpg,0,0.886043,0.464584,0.559896,0.162037,0.236111
747,24-11-2021_04-18-24_AM.jpg,0,0.905898,0.591146,0.705730,0.202777,0.287037
748,24-11-2021_04-18-24_AM.jpg,0,0.908115,0.809897,0.936980,0.276852,0.387962


In [144]:
simple_solution30.to_csv("simple_solution_30.csv", sep=';', index=False)

# Индекс 1 - head

In [50]:
head_labels_only = [file_name for file_name in os.listdir('train/labels') if 'head..txt' in file_name]
print(len(head_labels_only))

1741


In [51]:
head_images_names = [file_name.split('_head..txt')[0] for file_name in head_labels_only]
print(len(head_images_names))

1741


In [52]:
train_images_names, test_images_names = train_test_split(head_images_names, test_size=0.2, random_state=22)
print(len(train_images_names), len(test_images_names))

1392 349


In [53]:
#создадим обучающий датасет в формате yolo
!mkdir data_for_yolo/data_head
!mkdir data_for_yolo/data_head/images
!mkdir data_for_yolo/data_head/labels
!mkdir data_for_yolo/data_head/images/train
!mkdir data_for_yolo/data_head/labels/train
!mkdir data_for_yolo/data_head/images/test
!mkdir data_for_yolo/data_head/labels/test

In [54]:
#train
for train_image in train_images_names:
    #labels
    for file in os.listdir('train/labels'): 
        if '_head..txt' in file and train_image in file: 
            shutil.copy('train/labels/' + file, 'data_for_yolo/data_head/labels/train/' + file.replace('_head..txt', '.txt'))
        #images 
    for file in os.listdir('train/images'): 
        if train_image + '.jpg' == file: 
            shutil.copy('train/images/' + file, 'data_for_yolo/data_head/images/train/')

#test
for test_image in test_images_names:
    #labels
    for file in os.listdir('train/labels'): 
        if '_head..txt' in file and test_image in file: 
            shutil.copy('train/labels/' + file, 'data_for_yolo/data_head/labels/test/' + file.replace('_head..txt', '.txt'))
    #images 
    for file in os.listdir('train/images'): 
        if test_image + '.jpg' == file: 
            shutil.copy('train/images/' + file, 'data_for_yolo/data_head/images/test/')

In [55]:
#валидация 
print(len(os.listdir('data_for_yolo/data_head/images/test')))
print(len(os.listdir('data_for_yolo/data_head/images/train')))

349
1392


In [56]:
print(len(os.listdir('data_for_yolo/data_head/labels/test')))
print(len(os.listdir('data_for_yolo/data_head/labels/train')))

349
1392


## Обучение - head (1)

In [57]:
path_to_data = 'dataset_head.yaml'

In [58]:
!cd yolov5 && python3 train.py --img 416 --batch 16 --epochs 100 --data {path_to_data} --cfg models/yolov5s.yaml --name yolov5s_results_head --cache

train: weights=yolov5s.pt, cfg=models/yolov5s.yaml, data=dataset_head.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=yolov5s_results_head, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 20 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.1-328-ge309a85 Python-3.7.11 torch-1.7.0 CUDA:0 (GeForce GTX 1050 Ti, 4042MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5,

      5/99      1.7G    0.1267  0.008652         0        48       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        349        953      0.173       0.14     0.0595     0.0127

     Epoch   gpu_mem       box       obj       cls    labels  img_size
      6/99      1.7G     0.124  0.009056         0        47       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        349        953      0.259      0.161     0.0974     0.0219

     Epoch   gpu_mem       box       obj       cls    labels  img_size
      7/99      1.7G     0.123  0.009444         0        39       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        349        953      0.296      0.184      0.108     0.0258

     Epoch   gpu_mem       box       obj       cls    labels  img_size
      8/99      1.7G    0.1213  0.009


     Epoch   gpu_mem       box       obj       cls    labels  img_size
     31/99      1.7G    0.1043   0.01022         0        39       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        349        953      0.535      0.235      0.232     0.0636

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     32/99      1.7G    0.1043   0.01044         0        46       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        349        953      0.513      0.238      0.235     0.0629

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     33/99      1.7G    0.1015   0.01029         0        23       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        349        953      0.515      0.253      0.239     0.0635

     Epoch   gpu_mem       box      

               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        349        953      0.589      0.283      0.288     0.0851

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     57/99      1.7G   0.09571   0.01028         0        40       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        349        953      0.609       0.27      0.281     0.0855

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     58/99      1.7G   0.09302   0.01056         0        45       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        349        953      0.567       0.29      0.287      0.087

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     59/99      1.7G   0.09616    0.0105         0        48       416: 100%|███
               Class     Images     L


     Epoch   gpu_mem       box       obj       cls    labels  img_size
     82/99      1.7G   0.08856   0.01003         0        33       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        349        953      0.592      0.298      0.298     0.0919

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     83/99      1.7G    0.0863   0.01022         0        41       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        349        953      0.633      0.295      0.301     0.0946

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     84/99      1.7G   0.08732   0.01002         0        49       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        349        953      0.559      0.298      0.288     0.0911

     Epoch   gpu_mem       box      

In [145]:
# Поправить yolov5s_results_head ?
!cd yolov5 && python3 detect.py --weights runs/train/yolov5s_results_head/weights/best.pt --img 416 --conf 0.4 --source ../test/images --save-txt --save-conf

detect: weights=['runs/train/yolov5s_results_head/weights/best.pt'], source=../test/images, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-328-ge309a85 Python-3.7.11 torch-1.7.0 CUDA:0 (GeForce GTX 1050 Ti, 4042MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/0_23-11-2021_02-18-50_PM.jpg: 288x416 Done. (0.009s)
image 2/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/0_23-11-2021_02-18-56_PM.jpg: 352x416 1 head, Done. (0.009s)
image 3/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI

image 56/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/0_25-11-2021_09-27-14_AM.jpg: 288x416 Done. (0.008s)
image 57/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/0_25-11-2021_09-46-26_AM.jpg: 320x416 Done. (0.008s)
image 58/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/0_25-11-2021_10-03-07_AM.jpg: 288x416 Done. (0.008s)
image 59/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/0_25-11-2021_10-03-50_AM.jpg: 224x416 Done. (0.010s)
image 60/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/0_25-11-2021_10-55-30_AM.jpg: 352x416 1 head, Done. (0.009s)
image 61/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/0_25-11-2021_11-14-06_AM.jpg: 352x416 Done. (0.008s)
image 62/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/0_26-11-2021_02-25-08_AM.jpg: 320x416

image 113/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_02-37-22_PM.jpg: 256x416 1 head, Done. (0.008s)
image 114/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_02-37-31_PM.jpg: 256x416 1 head, Done. (0.007s)
image 115/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_02-37-35_PM.jpg: 256x416 1 head, Done. (0.008s)
image 116/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_02-37-57_PM.jpg: 256x416 3 heads, Done. (0.007s)
image 117/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_02-38-55_PM.jpg: 256x416 6 heads, Done. (0.009s)
image 118/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_02-39-07_PM.jpg: 320x416 1 head, Done. (0.010s)
image 119/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/

image 169/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_05-35-51_PM.jpg: 256x416 Done. (0.007s)
image 170/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_05-36-00_PM.jpg: 256x416 1 head, Done. (0.008s)
image 171/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_05-36-22_PM.jpg: 256x416 Done. (0.007s)
image 172/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_05-37-26_PM.jpg: 256x416 2 heads, Done. (0.007s)
image 173/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_05-38-10_PM.jpg: 256x416 2 heads, Done. (0.007s)
image 174/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_05-55-09_PM.jpg: 256x416 Done. (0.007s)
image 175/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_05-55-34_PM.j

image 225/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/24-11-2021_02-31-12_AM.jpg: 256x416 Done. (0.007s)
image 226/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/24-11-2021_02-32-58_AM.jpg: 256x416 Done. (0.008s)
image 227/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/24-11-2021_02-33-05_AM.jpg: 256x416 1 head, Done. (0.008s)
image 228/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/24-11-2021_02-42-12_PM.jpg: 256x416 3 heads, Done. (0.008s)
image 229/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/24-11-2021_02-42-46_PM.jpg: 256x416 1 head, Done. (0.008s)
image 230/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/24-11-2021_02-42-57_PM.jpg: 256x416 1 head, Done. (0.007s)
image 231/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/24-11-2021_02-43-

...

image 613/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_07-57-05_AM.jpg: 256x416 1 head, Done. (0.009s)
image 614/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_07-57-51_AM.jpg: 256x416 2 heads, Done. (0.012s)
image 615/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_07-57-57_AM.jpg: 256x416 1 head, Done. (0.010s)
image 616/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_07-58-31_AM.jpg: 256x416 4 heads, Done. (0.011s)
image 617/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_07-58-51_AM.jpg: 256x416 1 head, Done. (0.009s)
image 618/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_08-08-43_PM.jpg: 256x416 2 heads, Done. (0.010s)
image 619/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images

image 668/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_10-03-12_AM.jpg: 256x416 1 head, Done. (0.009s)
image 669/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_10-03-33_AM.jpg: 256x416 2 heads, Done. (0.008s)
image 670/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_10-19-38_AM.jpg: 256x416 Done. (0.010s)
image 671/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_10-20-32_AM.jpg: 256x416 Done. (0.008s)
image 672/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_10-20-40_AM.jpg: 256x416 2 heads, Done. (0.009s)
image 673/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_10-21-06_AM.jpg: 256x416 Done. (0.008s)
image 674/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_10-21-41_AM.j

image 724/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_04-47-47_AM.jpg: 256x416 1 head, Done. (0.008s)
image 725/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_04-48-11_AM.jpg: 256x416 3 heads, Done. (0.009s)
image 726/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_04-48-23_AM.jpg: 256x416 1 head, Done. (0.009s)
image 727/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_04-49-01_AM.jpg: 256x416 2 heads, Done. (0.007s)
image 728/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_05-04-43_AM.jpg: 256x416 Done. (0.008s)
image 729/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_05-04-56_AM.jpg: 256x416 1 head, Done. (0.007s)
image 730/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-20

image 779/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_08-05-04_AM.jpg: 256x416 1 head, Done. (0.009s)
image 780/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_08-05-07_AM.jpg: 256x416 Done. (0.008s)
image 781/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_08-05-21_AM.jpg: 256x416 Done. (0.009s)
image 782/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_08-06-03_AM.jpg: 256x416 2 heads, Done. (0.009s)
image 783/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_08-06-23_AM.jpg: 256x416 Done. (0.010s)
image 784/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_08-22-11_AM.jpg: 256x416 Done. (0.009s)
image 785/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_08-22-27_AM.jpg: 256x4

image 834/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_11-44-35_AM.jpg: 256x416 2 heads, Done. (0.009s)
image 835/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_11-45-44_AM.jpg: 256x416 2 heads, Done. (0.008s)
image 836/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_11-46-34_AM.jpg: 256x416 3 heads, Done. (0.009s)
image 837/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_11-47-14_AM.jpg: 256x416 1 head, Done. (0.008s)
image 838/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_12-04-22_PM.jpg: 256x416 1 head, Done. (0.008s)
image 839/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_12-07-24_PM.jpg: 256x416 2 heads, Done. (0.008s)
image 840/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/image

In [146]:
# Поменять путь yolov5/runs/detect/exp2/labels/ !!!
simple_solution_head = get_soliton_labels_df('yolov5/runs/detect/exp9/labels/')
simple_solution_head = pd.DataFrame(simple_solution_head, columns=['ImageID', 'LabelName', 'Conf', 'XMin', 'XMax', 'YMin', 'YMax'])

In [147]:
simple_solution_head['LabelName'] = str(1)
simple_solution_head

,ImageID,LabelName,Conf,XMin,XMax,YMin,YMax
0,24-11-2021_11-08-42_AM.jpg,1,0.698434,0.279688,0.293750,0.252778,0.279630
1,23-11-2021_02-19-04_PM.jpg,1,0.647007,0.327604,0.338542,0.275000,0.295370
2,24-11-2021_02-43-19_PM.jpg,1,0.642999,0.187500,0.214410,0.298611,0.347223
3,0_23-11-2021_02-18-56_PM.jpg,1,0.463836,0.883333,0.929167,0.410000,0.470000
4,24-11-2021_03-36-34_PM.jpg,1,0.718631,0.577009,0.591145,0.605921,0.631579
...,...,...,...,...,...,...,...
926,25-11-2021_04-18-04_PM.jpg,1,0.722557,0.821354,0.842188,0.507408,0.544444
927,24-11-2021_12-20-59_PM.jpg,1,0.564891,0.412500,0.419270,0.127778,0.141666
928,24-11-2021_12-20-59_PM.jpg,1,0.742034,0.030729,0.045833,0.208333,0.236111
929,26-11-2021_04-12-57_AM.jpg,1,0.755257,0.350521,0.370833,0.406481,0.442593


In [148]:
len(simple_solution30)

8808

In [149]:
simple_solution301 = simple_solution30.append(simple_solution_head, ignore_index = True)

In [150]:
len(simple_solution301)

9739

In [65]:
simple_solution301[3840:3855]

,ImageID,LabelName,Conf,XMin,XMax,YMin,YMax
3840,24-11-2021_11-26-12_AM.jpg,0,0.763747,0.274479,0.344791,0.096296,0.142592
3841,24-11-2021_11-26-12_AM.jpg,0,0.896054,0.462500,0.568229,0.165741,0.241667
3842,24-11-2021_11-26-12_AM.jpg,0,0.947891,0.686979,0.808333,0.486111,0.709259
3843,25-11-2021_01-56-24_PM.jpg,1,0.709646,0.102083,0.110937,0.503704,0.521296
3844,24-11-2021_05-31-33_AM.jpg,1,0.512072,0.418229,0.426041,0.375000,0.388888
3845,24-11-2021_05-31-33_AM.jpg,1,0.768985,0.892188,0.901042,0.581482,0.599074
3846,26-11-2021_10-46-49_AM.jpg,1,0.723787,0.040551,0.053571,0.526316,0.551316
3847,26-11-2021_10-46-49_AM.jpg,1,0.731074,0.766742,0.780506,0.094079,0.121053
3848,26-11-2021_10-46-49_AM.jpg,1,0.744966,0.532366,0.547991,0.305921,0.335527
3849,26-11-2021_10-46-49_AM.jpg,1,0.782965,0.595239,0.609003,0.255921,0.280921


In [66]:
simple_solution301.to_csv('simple_solution301.csv')

# Индекс 2 - face

In [67]:
face_labels_only = [file_name for file_name in os.listdir('train/labels') if 'face..txt' in file_name]
print(len(face_labels_only))

630


In [68]:
face_images_names = [file_name.split('_face..txt')[0] for file_name in face_labels_only]
print(len(face_images_names))

630


In [69]:
train_images_names, test_images_names = train_test_split(face_images_names, test_size=0.2, random_state=22)
print(len(train_images_names), len(test_images_names))

504 126


In [70]:
#создадим обучающий датасет в формате yolo
!mkdir data_for_yolo/data_face
!mkdir data_for_yolo/data_face/images
!mkdir data_for_yolo/data_face/labels
!mkdir data_for_yolo/data_face/images/train
!mkdir data_for_yolo/data_face/labels/train
!mkdir data_for_yolo/data_face/images/test
!mkdir data_for_yolo/data_face/labels/test

In [71]:
#train
for train_image in train_images_names:
    #labels
    for file in os.listdir('train/labels'): 
        if '_face..txt' in file and train_image in file: 
            shutil.copy('train/labels/' + file, 'data_for_yolo/data_face/labels/train/' + file.replace('_face..txt', '.txt'))
        #images 
    for file in os.listdir('train/images'): 
        if train_image + '.jpg' == file: 
            shutil.copy('train/images/' + file, 'data_for_yolo/data_face/images/train/')

#test
for test_image in test_images_names:
    #labels
    for file in os.listdir('train/labels'): 
        if '_face..txt' in file and test_image in file: 
            shutil.copy('train/labels/' + file, 'data_for_yolo/data_face/labels/test/' + file.replace('_face..txt', '.txt'))
    #images 
    for file in os.listdir('train/images'): 
        if test_image + '.jpg' == file: 
            shutil.copy('train/images/' + file, 'data_for_yolo/data_face/images/test/')

In [72]:
#валидация 
print(len(os.listdir('data_for_yolo/data_face/images/test')))
print(len(os.listdir('data_for_yolo/data_face/images/train')))

126
504


In [73]:
print(len(os.listdir('data_for_yolo/data_face/labels/test')))
print(len(os.listdir('data_for_yolo/data_face/labels/train')))

126
504


## Обучение - face (2)

In [74]:
path_to_data = 'dataset_face.yaml'

In [75]:
!cd yolov5 && python3 train.py --img 416 --batch 16 --epochs 100 --data {path_to_data} --cfg models/yolov5s.yaml --name yolov5s_results_face --cache

train: weights=yolov5s.pt, cfg=models/yolov5s.yaml, data=dataset_face.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=yolov5s_results_face, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 20 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.1-328-ge309a85 Python-3.7.11 torch-1.7.0 CUDA:0 (GeForce GTX 1050 Ti, 4042MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5,

      5/99     1.71G    0.1427  0.003918         0         7       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        126        162    0.00166      0.154    0.00155   0.000373

     Epoch   gpu_mem       box       obj       cls    labels  img_size
      6/99     1.71G    0.1395  0.004462         0         9       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        126        162    0.00137       0.21    0.00635    0.00225

     Epoch   gpu_mem       box       obj       cls    labels  img_size
      7/99     1.71G    0.1393  0.004234         0         8       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        126        162      0.153    0.00617     0.0105    0.00338

     Epoch   gpu_mem       box       obj       cls    labels  img_size
      8/99     1.71G    0.1359   0.00


     Epoch   gpu_mem       box       obj       cls    labels  img_size
     31/99     1.71G    0.1178  0.005949         0         5       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        126        162       0.19      0.123     0.0492    0.00826

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     32/99     1.71G    0.1188    0.0058         0         8       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        126        162      0.227       0.13     0.0808     0.0201

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     33/99     1.71G    0.1209  0.005697         0         9       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        126        162      0.195      0.117     0.0766     0.0202

     Epoch   gpu_mem       box      

               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        126        162      0.285      0.167      0.103      0.024

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     57/99     1.71G    0.1044  0.006179         0        12       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        126        162      0.263      0.136     0.0861     0.0229

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     58/99     1.71G    0.1064  0.006051         0        10       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        126        162      0.226      0.173      0.113     0.0292

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     59/99     1.71G    0.1027  0.006397         0         6       416: 100%|███
               Class     Images     L


     Epoch   gpu_mem       box       obj       cls    labels  img_size
     82/99     1.71G   0.08933  0.006529         0         9       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        126        162      0.298       0.16      0.138     0.0374

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     83/99     1.71G   0.08703  0.006599         0         9       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        126        162       0.18      0.265      0.138     0.0355

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     84/99     1.71G   0.09163  0.006535         0         8       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all        126        162      0.273      0.216      0.145     0.0409

     Epoch   gpu_mem       box      

In [151]:
# Поправить yolov5s_results_face ?
!cd yolov5 && python3 detect.py --weights runs/train/yolov5s_results_face/weights/best.pt --img 416 --conf 0.4 --source ../test/images --save-txt --save-conf

detect: weights=['runs/train/yolov5s_results_face/weights/best.pt'], source=../test/images, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-328-ge309a85 Python-3.7.11 torch-1.7.0 CUDA:0 (GeForce GTX 1050 Ti, 4042MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/0_23-11-2021_02-18-50_PM.jpg: 288x416 Done. (0.009s)
image 2/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/0_23-11-2021_02-18-56_PM.jpg: 352x416 Done. (0.009s)
image 3/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Та

image 55/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/0_25-11-2021_08-50-08_AM.jpg: 288x416 1 face, Done. (0.008s)
image 56/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/0_25-11-2021_09-27-14_AM.jpg: 288x416 Done. (0.008s)
image 57/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/0_25-11-2021_09-46-26_AM.jpg: 320x416 Done. (0.008s)
image 58/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/0_25-11-2021_10-03-07_AM.jpg: 288x416 Done. (0.009s)
image 59/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/0_25-11-2021_10-03-50_AM.jpg: 224x416 Done. (0.008s)
image 60/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/0_25-11-2021_10-55-30_AM.jpg: 352x416 Done. (0.008s)
image 61/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/0_25-11-2021_11-14-06_AM.jpg: 352x416

image 112/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_02-37-17_PM.jpg: 256x416 Done. (0.009s)
image 113/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_02-37-22_PM.jpg: 256x416 Done. (0.010s)
image 114/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_02-37-31_PM.jpg: 256x416 Done. (0.008s)
image 115/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_02-37-35_PM.jpg: 256x416 Done. (0.008s)
image 116/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_02-37-57_PM.jpg: 256x416 2 faces, Done. (0.008s)
image 117/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_02-38-55_PM.jpg: 256x416 1 face, Done. (0.010s)
image 118/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_02-39-07_PM.jpg: 320x4

image 169/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_05-35-51_PM.jpg: 256x416 Done. (0.009s)
image 170/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_05-36-00_PM.jpg: 256x416 Done. (0.008s)
image 171/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_05-36-22_PM.jpg: 256x416 Done. (0.007s)
image 172/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_05-37-26_PM.jpg: 256x416 Done. (0.008s)
image 173/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_05-38-10_PM.jpg: 256x416 Done. (0.007s)
image 174/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_05-55-09_PM.jpg: 256x416 Done. (0.007s)
image 175/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_05-55-34_PM.jpg: 256x416 Done. (0.008s)

image 226/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/24-11-2021_02-32-58_AM.jpg: 256x416 Done. (0.007s)
image 227/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/24-11-2021_02-33-05_AM.jpg: 256x416 Done. (0.008s)
image 228/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/24-11-2021_02-42-12_PM.jpg: 256x416 Done. (0.007s)
image 229/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/24-11-2021_02-42-46_PM.jpg: 256x416 Done. (0.009s)
image 230/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/24-11-2021_02-42-57_PM.jpg: 256x416 Done. (0.008s)
image 231/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/24-11-2021_02-43-15_PM.jpg: 256x416 Done. (0.008s)
image 232/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/24-11-2021_02-43-19_PM.jpg: 256x416 1 face, Done. 

...

image 626/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_08-32-04_AM.jpg: 256x416 Done. (0.008s)
image 627/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_08-32-41_AM.jpg: 256x416 Done. (0.008s)
image 628/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_08-32-54_AM.jpg: 256x416 Done. (0.010s)
image 629/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_08-33-28_AM.jpg: 256x416 Done. (0.010s)
image 630/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_08-33-34_AM.jpg: 256x416 Done. (0.008s)
image 631/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_08-33-38_AM.jpg: 256x416 Done. (0.008s)
image 632/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_08-33-52_AM.jpg: 256x416 1 face, Done. 

image 683/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_10-56-11_AM.jpg: 256x416 Done. (0.011s)
image 684/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_10-56-23_AM.jpg: 256x416 Done. (0.009s)
image 685/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_10-56-27_AM.jpg: 256x416 Done. (0.008s)
image 686/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_10-56-36_AM.jpg: 256x416 Done. (0.008s)
image 687/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_10-57-03_AM.jpg: 256x416 Done. (0.009s)
image 688/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_10-57-24_AM.jpg: 256x416 1 face, Done. (0.009s)
image 689/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_11-14-06_AM.jpg: 256x416 Done. 

image 740/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_06-00-53_AM.jpg: 320x416 Done. (0.008s)
image 741/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_06-16-35_AM.jpg: 256x416 Done. (0.008s)
image 742/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_06-17-09_AM.jpg: 256x416 Done. (0.008s)
image 743/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_06-17-16_AM.jpg: 256x416 1 face, Done. (0.008s)
image 744/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_06-18-20_AM.jpg: 256x416 Done. (0.008s)
image 745/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_06-18-28_AM.jpg: 256x416 Done. (0.007s)
image 746/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_06-18-42_AM.jpg: 256x416 Done. 

image 797/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_09-16-58_AM.jpg: 256x416 Done. (0.008s)
image 798/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_09-17-10_AM.jpg: 256x416 Done. (0.009s)
image 799/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_09-17-12_AM.jpg: 256x416 1 face, Done. (0.007s)
image 800/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_09-17-30_AM.jpg: 256x416 Done. (0.009s)
image 801/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_09-17-48_AM.jpg: 256x416 Done. (0.009s)
image 802/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_09-33-08_AM.jpg: 256x416 1 face, Done. (0.010s)
image 803/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_09-33-44_AM.jpg: 256x41

image 854/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/3_24-11-2021_04-17-46_AM.jpg: 256x416 Done. (0.009s)
image 855/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/3_25-11-2021_04-21-55_AM.jpg: 416x352 Done. (0.009s)
Speed: 0.3ms pre-process, 8.3ms inference, 0.4ms NMS per image at shape (1, 3, 416, 416)
Results saved to runs/detect/exp10
136 labels saved to runs/detect/exp10/labels


In [152]:
# Поменять путь yolov5/runs/detect/exp2/labels/ !!!
simple_solution_face = get_soliton_labels_df('yolov5/runs/detect/exp10/labels/')
simple_solution_face = pd.DataFrame(simple_solution_face, columns=['ImageID', 'LabelName', 'Conf', 'XMin', 'XMax', 'YMin', 'YMax'])

In [153]:
simple_solution_face['LabelName'] = str(2)
simple_solution_face

,ImageID,LabelName,Conf,XMin,XMax,YMin,YMax
0,24-11-2021_11-08-42_AM.jpg,2,0.683161,0.282812,0.291146,0.270371,0.287037
1,24-11-2021_02-43-19_PM.jpg,2,0.492128,0.191841,0.203559,0.327161,0.350309
2,23-11-2021_02-56-18_PM.jpg,2,0.400344,0.473959,0.480655,0.228289,0.242763
3,26-11-2021_03-53-42_AM.jpg,2,0.708715,0.219122,0.234376,0.465131,0.495395
4,26-11-2021_06-54-45_AM.jpg,2,0.552468,0.437500,0.446354,0.671297,0.688889
...,...,...,...,...,...,...,...
145,26-11-2021_04-48-23_AM.jpg,2,0.507533,0.523959,0.534375,0.452778,0.472222
146,24-11-2021_09-04-10_AM.jpg,2,0.551908,0.268602,0.278274,0.623684,0.642106
147,24-11-2021_01-32-29_PM.jpg,2,0.614391,0.449480,0.462500,0.646296,0.673148
148,24-11-2021_12-20-59_PM.jpg,2,0.425473,0.032812,0.039583,0.221297,0.235185


In [154]:
len(simple_solution301)

9739

In [155]:
simple_solution3012 = simple_solution301.append(simple_solution_face, ignore_index = True)

In [156]:
len(simple_solution3012)

9889

In [83]:
simple_solution3012.to_csv('simple_solution3012.csv')

# Индекс 4 - carplate

In [84]:
carplate_labels_only = [file_name for file_name in os.listdir('train/labels') if 'carplate..txt' in file_name]
print(len(carplate_labels_only))

246


In [85]:
carplate_images_names = [file_name.split('_carplate..txt')[0] for file_name in carplate_labels_only]
print(len(carplate_images_names))

246


In [86]:
train_images_names, test_images_names = train_test_split(carplate_images_names, test_size=0.2, random_state=22)
print(len(train_images_names), len(test_images_names))

196 50


In [87]:
#создадим обучающий датасет в формате yolo
!mkdir data_for_yolo/data_carplate
!mkdir data_for_yolo/data_carplate/images
!mkdir data_for_yolo/data_carplate/labels
!mkdir data_for_yolo/data_carplate/images/train
!mkdir data_for_yolo/data_carplate/labels/train
!mkdir data_for_yolo/data_carplate/images/test
!mkdir data_for_yolo/data_carplate/labels/test

In [88]:
#train
for train_image in train_images_names:
    #labels
    for file in os.listdir('train/labels'): 
        if '_carplate..txt' in file and train_image in file: 
            shutil.copy('train/labels/' + file, 'data_for_yolo/data_carplate/labels/train/' + file.replace('_carplate..txt', '.txt'))
        #images 
    for file in os.listdir('train/images'): 
        if train_image + '.jpg' == file: 
            shutil.copy('train/images/' + file, 'data_for_yolo/data_carplate/images/train/')

#test
for test_image in test_images_names:
    #labels
    for file in os.listdir('train/labels'): 
        if '_carplate..txt' in file and test_image in file: 
            shutil.copy('train/labels/' + file, 'data_for_yolo/data_carplate/labels/test/' + file.replace('_carplate..txt', '.txt'))
    #images 
    for file in os.listdir('train/images'): 
        if test_image + '.jpg' == file: 
            shutil.copy('train/images/' + file, 'data_for_yolo/data_carplate/images/test/')

In [89]:
#валидация 
print(len(os.listdir('data_for_yolo/data_carplate/images/test')))
print(len(os.listdir('data_for_yolo/data_carplate/images/train')))

50
196


In [91]:
print(len(os.listdir('data_for_yolo/data_carplate/labels/test')))
print(len(os.listdir('data_for_yolo/data_carplate/labels/train')))

50
196


## Обучение - carplate (4)

In [92]:
path_to_data = 'dataset_carplate.yaml'

In [93]:
!cd yolov5 && python3 train.py --img 416 --batch 16 --epochs 100 --data {path_to_data} --cfg models/yolov5s.yaml --name yolov5s_results_carplate --cache

train: weights=yolov5s.pt, cfg=models/yolov5s.yaml, data=dataset_carplate.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=yolov5s_results_carplate, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 7 (delta 4), reused 7 (delta 4), pack-reused 0
Распаковка объектов: 100% (7/7), 1.19 КиБ | 305.00 КиБ/с, готово.
Из https://github.com/ultralytics/yolov5
   59b2353..ebb9c99  classifier -> origin/classifier
github: ⚠️ YO

      6/99     1.62G   0.08138   0.01574         0         6       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all         50         50      0.776       0.12      0.348     0.0856

     Epoch   gpu_mem       box       obj       cls    labels  img_size
      7/99     1.62G   0.08009   0.01537         0         6       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all         50         50      0.521       0.14       0.25     0.0447

     Epoch   gpu_mem       box       obj       cls    labels  img_size
      8/99     1.62G    0.0695   0.01725         0         7       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all         50         50      0.619       0.26      0.428      0.146

     Epoch   gpu_mem       box       obj       cls    labels  img_size
      9/99     1.62G   0.06854   0.01


     Epoch   gpu_mem       box       obj       cls    labels  img_size
     32/99     1.62G   0.03172  0.009275         0        10       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all         50         50      0.997          1      0.995      0.564

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     33/99     1.62G    0.0295  0.008257         0        11       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all         50         50      0.999          1      0.995      0.665

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     34/99     1.62G   0.02947  0.007638         0         7       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all         50         50      0.999          1      0.995      0.631

     Epoch   gpu_mem       box      

               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all         50         50      0.999          1      0.995      0.751

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     58/99     1.62G   0.02236  0.005951         0         6       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all         50         50      0.999          1      0.995      0.738

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     59/99     1.62G   0.02507  0.005604         0         5       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all         50         50      0.998          1      0.995      0.748

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     60/99     1.62G   0.02326  0.006238         0         5       416: 100%|███
               Class     Images     L


     Epoch   gpu_mem       box       obj       cls    labels  img_size
     83/99     1.62G   0.01545  0.004753         0         3       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all         50         50      0.999          1      0.995      0.808

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     84/99     1.62G   0.01427  0.005191         0         9       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all         50         50      0.999          1      0.995       0.81

     Epoch   gpu_mem       box       obj       cls    labels  img_size
     85/99     1.62G   0.01582  0.004812         0         5       416: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
                 all         50         50      0.999          1      0.995      0.815

     Epoch   gpu_mem       box      

In [157]:
# Поправить yolov5s_results_carplate ?
!cd yolov5 && python3 detect.py --weights runs/train/yolov5s_results_carplate/weights/best.pt --img 416 --conf 0.4 --source ../test/images --save-txt --save-conf

detect: weights=['runs/train/yolov5s_results_carplate/weights/best.pt'], source=../test/images, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 v6.1-328-ge309a85 Python-3.7.11 torch-1.7.0 CUDA:0 (GeForce GTX 1050 Ti, 4042MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/0_23-11-2021_02-18-50_PM.jpg: 288x416 1 carplate, Done. (0.009s)
image 2/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/0_23-11-2021_02-18-56_PM.jpg: 352x416 1 carplate, Done. (0.010s)
image 3/855 /home/molodec/BIG_DA

image 52/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/0_25-11-2021_07-58-20_AM.jpg: 288x416 1 carplate, Done. (0.011s)
image 53/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/0_25-11-2021_08-09-32_PM.jpg: 320x416 1 carplate, Done. (0.008s)
image 54/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/0_25-11-2021_08-34-03_AM.jpg: 320x416 1 carplate, Done. (0.009s)
image 55/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/0_25-11-2021_08-50-08_AM.jpg: 288x416 1 carplate, Done. (0.008s)
image 56/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/0_25-11-2021_09-27-14_AM.jpg: 288x416 1 carplate, Done. (0.008s)
image 57/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/0_25-11-2021_09-46-26_AM.jpg: 320x416 1 carplate, Done. (0.012s)
image 58/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татар

image 105/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_02-19-04_PM.jpg: 256x416 1 carplate, Done. (0.008s)
image 106/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_02-19-26_PM.jpg: 256x416 Done. (0.008s)
image 107/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_02-19-42_PM.jpg: 256x416 Done. (0.008s)
image 108/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_02-20-25_PM.jpg: 256x416 Done. (0.007s)
image 109/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_02-21-03_PM.jpg: 256x416 Done. (0.008s)
image 110/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_02-21-18_PM.jpg: 256x416 1 carplate, Done. (0.008s)
image 111/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_02-36-28_PM.jpg

image 162/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_05-01-36_PM.jpg: 256x416 1 carplate, Done. (0.009s)
image 163/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_05-01-38_PM.jpg: 256x416 Done. (0.008s)
image 164/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_05-02-13_PM.jpg: 256x416 Done. (0.010s)
image 165/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_05-02-16_PM.jpg: 256x416 Done. (0.007s)
image 166/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_05-18-06_PM.jpg: 256x416 Done. (0.010s)
image 167/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_05-18-46_PM.jpg: 256x416 Done. (0.010s)
image 168/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/23-11-2021_05-19-24_PM.jpg: 256x416 Do

image 219/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/24-11-2021_02-25-19_PM.jpg: 256x416 Done. (0.009s)
image 220/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/24-11-2021_02-25-31_PM.jpg: 256x416 Done. (0.008s)
image 221/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/24-11-2021_02-25-34_PM.jpg: 256x416 Done. (0.008s)
image 222/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/24-11-2021_02-25-41_PM.jpg: 256x416 Done. (0.008s)
image 223/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/24-11-2021_02-25-48_PM.jpg: 256x416 1 carplate, Done. (0.008s)
image 224/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/24-11-2021_02-31-10_AM.jpg: 256x416 Done. (0.007s)
image 225/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/24-11-2021_02-31-12_AM.jpg: 256x416 Do

...

image 618/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_08-08-43_PM.jpg: 256x416 Done. (0.008s)
image 619/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_08-14-18_AM.jpg: 256x416 Done. (0.008s)
image 620/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_08-14-36_AM.jpg: 256x416 Done. (0.008s)
image 621/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_08-14-45_AM.jpg: 256x416 Done. (0.008s)
image 622/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_08-15-01_AM.jpg: 256x416 Done. (0.008s)
image 623/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_08-15-15_AM.jpg: 256x416 Done. (0.007s)
image 624/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_08-16-05_AM.jpg: 256x416 1 carplate, Do

image 675/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_10-21-45_AM.jpg: 256x416 1 carplate, Done. (0.008s)
image 676/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_10-38-15_AM.jpg: 256x416 Done. (0.008s)
image 677/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_10-38-29_AM.jpg: 256x416 Done. (0.008s)
image 678/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_10-38-33_AM.jpg: 256x416 Done. (0.009s)
image 679/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_10-38-55_AM.jpg: 256x416 Done. (0.009s)
image 680/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_10-39-17_AM.jpg: 256x416 Done. (0.009s)
image 681/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/25-11-2021_10-55-24_AM.jpg: 256x416 Do

image 732/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_05-23-48_AM.jpg: 256x416 Done. (0.008s)
image 733/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_05-24-06_AM.jpg: 256x416 Done. (0.009s)
image 734/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_05-24-38_AM.jpg: 256x416 Done. (0.008s)
image 735/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_05-40-18_AM.jpg: 256x416 Done. (0.009s)
image 736/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_05-40-45_AM.jpg: 256x416 Done. (0.008s)
image 737/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_05-42-17_AM.jpg: 256x416 Done. (0.009s)
image 738/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_05-42-59_AM.jpg: 256x416 Done. (0.009s)

image 789/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_08-40-11_AM.jpg: 256x416 Done. (0.007s)
image 790/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_08-40-24_AM.jpg: 256x416 Done. (0.009s)
image 791/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_08-41-58_AM.jpg: 320x416 Done. (0.008s)
image 792/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_08-58-00_AM.jpg: 256x416 Done. (0.008s)
image 793/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_08-58-03_AM.jpg: 256x416 Done. (0.007s)
image 794/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_08-58-53_AM.jpg: 256x416 1 carplate, Done. (0.007s)
image 795/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_08-59-28_AM.jpg: 256x416 Do

image 846/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/26-11-2021_12-47-01_PM.jpg: 256x416 Done. (0.009s)
image 847/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/2_23-11-2021_02-20-07_PM.jpg: 256x416 1 carplate, Done. (0.008s)
image 848/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/2_24-11-2021_10-14-19_AM.jpg: 224x416 1 carplate, Done. (0.008s)
image 849/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/2_24-11-2021_11-26-05_AM.jpg: 384x416 1 carplate, Done. (0.010s)
image 850/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/2_25-11-2021_05-15-29_AM.jpg: 256x416 1 carplate, Done. (0.009s)
image 851/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Татарстан/test/images/2_25-11-2021_07-04-34_AM.jpg: 416x352 1 carplate, Done. (0.009s)
image 852/855 /home/molodec/BIG_DATA/КОНКУРСЫ/Hacks_AI_2022/Татарстан/Та

In [158]:
# Поменять путь yolov5/runs/detect/exp2/labels/ !!!
simple_solution_carplate = get_soliton_labels_df('yolov5/runs/detect/exp11/labels/')
simple_solution_carplate = pd.DataFrame(simple_solution_carplate, columns=['ImageID', 'LabelName', 'Conf', 'XMin', 'XMax', 'YMin', 'YMax'])

In [159]:
simple_solution_carplate['LabelName'] = str(4)
simple_solution_carplate

,ImageID,LabelName,Conf,XMin,XMax,YMin,YMax
0,24-11-2021_11-08-42_AM.jpg,4,0.564101,0.000521,0.047917,2.000000e-07,0.108333
1,23-11-2021_02-19-04_PM.jpg,4,0.514021,0.528646,0.620312,8.018515e-01,0.962036
2,0_23-11-2021_02-18-56_PM.jpg,4,0.937285,0.520833,0.795833,7.350000e-01,0.850000
3,24-11-2021_01-31-29_PM.jpg,4,0.699467,0.832962,1.000001,9.236840e-01,0.985526
4,0_24-11-2021_03-54-02_PM.jpg,4,0.970710,0.080460,0.170498,3.881580e-01,0.552632
...,...,...,...,...,...,...,...
231,0_26-11-2021_09-35-12_AM.jpg,4,0.874784,0.175743,0.371287,8.181814e-01,0.909091
232,0_26-11-2021_10-27-24_AM.jpg,4,0.943676,0.127168,0.326590,7.549015e-01,0.875816
233,0_26-11-2021_08-39-44_AM.jpg,4,0.919444,0.166666,0.418440,7.449670e-01,0.855705
234,0_24-11-2021_05-12-35_AM.jpg,4,0.897528,0.089928,0.217626,4.756100e-01,0.641464


In [160]:
len(simple_solution_carplate)

236

In [161]:
simple_solution30124 = simple_solution3012.append(simple_solution_carplate, ignore_index = True)

In [162]:
len(simple_solution3012)

9889

In [163]:
len(simple_solution30124)

10125

In [164]:
simple_solution30124.to_csv("submit_01_2209.csv", sep=';', index=False)

In [165]:
df = pd.read_csv("submit_01_2209.csv", sep=';', index_col=None)

In [171]:
df[df['LabelName']==4]

,ImageID,LabelName,Conf,XMin,XMax,YMin,YMax
9889,24-11-2021_11-08-42_AM.jpg,4,0.564101,0.000521,0.047917,2.000000e-07,0.108333
9890,23-11-2021_02-19-04_PM.jpg,4,0.514021,0.528646,0.620312,8.018515e-01,0.962036
9891,0_23-11-2021_02-18-56_PM.jpg,4,0.937285,0.520833,0.795833,7.350000e-01,0.850000
9892,24-11-2021_01-31-29_PM.jpg,4,0.699467,0.832962,1.000001,9.236840e-01,0.985526
9893,0_24-11-2021_03-54-02_PM.jpg,4,0.970710,0.080460,0.170498,3.881580e-01,0.552632
...,...,...,...,...,...,...,...
10120,0_26-11-2021_09-35-12_AM.jpg,4,0.874784,0.175742,0.371287,8.181814e-01,0.909091
10121,0_26-11-2021_10-27-24_AM.jpg,4,0.943676,0.127168,0.326590,7.549015e-01,0.875816
10122,0_26-11-2021_08-39-44_AM.jpg,4,0.919444,0.166666,0.418440,7.449670e-01,0.855705
10123,0_24-11-2021_05-12-35_AM.jpg,4,0.897528,0.089928,0.217626,4.756100e-01,0.641464


In [107]:
type(df['LabelName'])

pandas.core.series.Series

In [109]:
df2 = pd.read_csv("sample_solution30124_2.csv", sep=';', index_col=None)

In [125]:
df2[df2['LabelName']==3

SyntaxError: unexpected EOF while parsing (239311177.py, line 1)

In [111]:
df2

,ImageID,LabelName,Conf,XMin,XMax,YMin,YMax
0,25-11-2021_10-20-38_AM.jpg,3,0.556868,0.086979,0.094792,0.212037,0.247223
1,25-11-2021_10-20-38_AM.jpg,3,0.728934,0.899479,0.917187,0.389815,0.453703
2,25-11-2021_10-20-38_AM.jpg,3,0.757751,0.878646,0.895834,0.352778,0.420370
3,25-11-2021_10-20-38_AM.jpg,3,0.809890,0.930208,0.944792,0.372222,0.441666
4,24-11-2021_10-50-24_AM.jpg,3,0.582441,0.020833,0.043229,0.408334,0.502778
...,...,...,...,...,...,...,...
4359,2_24-11-2021_04-17-33_AM.jpg,4,0.821122,0.602941,0.818627,0.746667,0.813333
4360,0_26-11-2021_11-17-01_AM.jpg,4,0.950545,0.891986,0.972126,0.730770,0.816568
4361,0_25-11-2021_01-06-25_AM.jpg,4,0.935347,0.621077,0.813902,0.556410,0.751282
4362,0_23-11-2021_03-50-12_PM.jpg,4,0.838751,0.146643,0.323321,0.633523,0.747159


'new_dir/file.txt'